In [3]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Define dataset path and image size
dataset_path = "data"  # Path to your dataset
image_size = (120, 160)  # Resize to 120x160 for Small-VGG

print('Our dataset = ')
print(os.listdir(dataset_path))

Our dataset = 
['Grupo_0', 'Grupo_1', 'Grupo_2', 'Grupo_3', 'Grupo_4', 'Grupo_5']


LG images

In [ ]:
# Function to load images from the LG folder (following the exact structure)
def load_images_from_folder(folder_path):
    images = []
    labels = []
    names = []
    corruptedFiles = []
    image_extensions = ["bmp"]

    # Traverse through the group -> subject -> LG -> session -> anotherfolder -> images
    for group_folder in os.listdir(folder_path):
        group_path = os.path.join(folder_path, group_folder)
        if os.path.isdir(group_path):  # If it is a group folder
            for subject_folder in os.listdir(group_path):
                subject_path = os.path.join(group_path, subject_folder)
                if os.path.isdir(subject_path):  # If it is a subject folder
                    lg_folder = os.path.join(subject_path, "LG")  # Looking specifically for the 'LG' folder
                    if os.path.isdir(lg_folder):  # Only process if 'LG' folder exists
                        # Iterate over sessions inside 'LG' folder
                        for session_folder in os.listdir(lg_folder):
                            session_path = os.path.join(lg_folder, session_folder)
                            if os.path.isdir(session_path):  # Check each session
                                # Iterate over each subfolder inside the session
                                for subfolder in os.listdir(session_path):
                                    subfolder_path = os.path.join(session_path, subfolder)
                                    if os.path.isdir(subfolder_path):  # If it's a subfolder containing images
                                        # Now we go through all the image files
                                        for file in os.listdir(subfolder_path):
                                            if any(file.lower().endswith(ext) for ext in image_extensions):
                                                img_path = os.path.join(subfolder_path, file)
                                                try:
                                                    # Read image in grayscale and resize
                                                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                                                    if img is not None:
                                                        img = cv2.resize(img, image_size)
                                                        images.append(img)
                                                        labels.append(file[6] != '0')  # Assuming the label is based on the file name (0 or 1)
                                                        names.append(file)
                                                except Exception as e:
                                                    corruptedFiles.append((img_path, str(e)))

    print(f"Following {len(corruptedFiles)} files are corrupt or encountered error: \n {corruptedFiles}")
    
    # Return the images, labels, and names if any images were found
    if images:
        return np.array(images), np.array(labels), np.array(names)
    else:
        print("No images were loaded.")
        return np.array([]), np.array([]), np.array([])  # Return empty arrays if no images are found

# Load dataset (only from the LG folder)
images, labels, names = load_images_from_folder(dataset_path)
print(f"Read {len(images)} images from the LG folder with shape {images.shape}")

Following 0 files are corrupt or encountered error: 
 []
Read 2820 images from the LG folder with shape (2820, 160, 120)


In [5]:
# Preprocess the data
images = images / 255.0  # Normalize pixel values
images = images.reshape(images.shape[0], image_size[0], image_size[1], 1)  # Add channel dimension
labels = to_categorical(labels, num_classes=2)  # Convert labels to one-hot encoding

# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.4, random_state=42)


In [6]:
# Define the Small-VGG model
def create_small_vgg():
    model = Sequential()
    # Block 1
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(image_size[0], image_size[1], 1)))
    model.add(MaxPooling2D((2, 2)))
    # Block 2
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    # Block 3
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    # Dense layers
    model.add(Flatten())
    model.add(Dense(7200, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  # Output layer for binary classification
    return model


In [ ]:
# Compile and train the Small-VGG model
model = create_small_vgg()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy: {:.2f}%".format(test_acc * 100))

c:\Users\Sneha Thakur\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 58s 661ms/step - accuracy: 0.8116 - loss: 0.6976 - val_accuracy: 0.7965 - val_loss: 0.5066
Epoch 2/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 54s 633ms/step - accuracy: 0.8111 - loss: 0.4922 - val_accuracy: 0.7965 - val_loss: 0.5268
Epoch 3/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 53s 627ms/step - accuracy: 0.7975 - loss: 0.5130 - val_accuracy: 0.7965 - val_loss: 0.5079
Epoch 4/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 56s 659ms/step - accuracy: 0.7937 - loss: 0.5151 - val_accuracy: 0.7965 - val_loss: 0.5061
Epoch 5/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 53s 626ms/step - accuracy: 0.8198 - loss: 0.4772 - val_accuracy: 0.7965 - val_loss: 0.5066
Epoch 6/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 54s 632ms/step - accuracy: 0.7967 - loss: 0.5051 - val_accuracy: 0.7965 - val_loss: 0.5049
Epoch 7/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 53s 621ms/step - accuracy: 0.8076 - loss: 0.4908 - val_accuracy: 0.7965 - val_loss: 0.5057
Epoch 8/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 53s 625ms/step - accuracy: 0.7999 - loss: 0.5045 - 

In [8]:
# Classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)
print("Classification Report:\n", classification_report(y_test_classes, y_pred_classes))


36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.46      0.57       247
           1       0.86      0.95      0.91       881

    accuracy                           0.85      1128
   macro avg       0.80      0.71      0.74      1128
weighted avg       0.84      0.85      0.83      1128

